psycopg2 is a PostgreSQL adapter for Python that provides a robust interface for interacting with PostgreSQL databaspsycopg2 is thread-safe at the connection level, which means multiple threads can use separate connections to the database safely.es. It adheres to the Python DB-API 2.0 specification, ensuring consistency and compatibility with Python's standard database interaction practices.  

psycopg2 is thread-safe at the connection level, which means multiple threads can use separate connections to the database safely.  

psycopg2 optimizes query execution, including support for server-side cursors, parameterized queries, and efficient handling of large datasets.  

psycopg2 offers connection pooling for efficient management of database connections, especially in applications with high concurrency  

In [ ]:
import psycopg2


In [82]:
#The connection to a PostgreSQL database instance is managed by the connection object.
#It encapsulates a database session, created using the function connect().

conn = psycopg2.connect(database="postgres",
                        host="localhost",
                        user="postgres",
                        password="ReportingUser",
                        port="5432")

 The cursor object makes it possible for Python scripts to run PostgreSQL commands within a database session. The connection generates cursors, then the cursor() method ties them permanently to the connection. All commands are carried out within the framework of the connection-enclosed database session.

In [83]:
# create a cursor object 

cursor = conn.cursor()


In [ ]:
# execute method is used to run a sql query 
# create schema
cursor.execute("create schema demo")

# # create a table
# cursor.execute("""CREATE TABLE demo.cars ( brand VARCHAR(255),
#                                    model VARCHAR(255),
#                                    year INT
#                                   )""")

In [ ]:
#nsert data in the table

cursor.execute("""INSERT INTO demo.cars (brand, model, year)
VALUES ('Ford', 'Mustang', 1964);""")

In [85]:
# commit the transaction
conn.commit()

In [ ]:
#fetch data from the table 
cursor.execute("select * from demo.cars")
rows = cursor.fetchall()
for row in rows:
    print(row)

In [86]:
# close the cursor and connection

cursor.close()
conn.close()

`autocommit`: Enables or disables autocommit mode.  
`commit()`: Commits the current transaction.  
`rollback()`: Rolls back the current transaction.  
`execute()`: Runs a SQL query.  
`fetchall()`: Retrieves all rows of a query result.  
`fetchone()`: Retrieves a single row of a query result.  
`fetchmany(size=n)`: Retrieves n rows from a query result.

In [94]:
import psycopg2
from psycopg2 import pool
import threading
import time

# Database connection parameters
db_config = {
    "user": "postgres",
    "password": "ReportingUser",
    "host": "localhost",
    "port": "5432",
    "database": "postgres"
}

# Create a connection pool with 5 connections
connection_pool = psycopg2.pool.SimpleConnectionPool(
    1,  # Minimum number of connections
    5,  # Maximum number of connections
    **db_config
)

# Function to simulate querying with multiple cursors using connection pooling
def query_with_pool_and_multiple_cursors(thread_id):
    # Get a connection from the pool
    conn = connection_pool.getconn()
    
    # Create two cursors on the same connection
    cursor1 = conn.cursor()
    cursor2 = conn.cursor()

    print(f"Thread-{thread_id} executing query...")

    # First query using the first cursor
    cursor1.execute("SELECT * FROM users LIMIT 5;")
    rows1 = cursor1.fetchall()

    # Second query using the second cursor
    cursor2.execute("SELECT * FROM orders LIMIT 5;")
    rows2 = cursor2.fetchall()

    # Print the results of both queries
    print(f"Thread-{thread_id} fetched {len(rows1)} rows from users and {len(rows2)} rows from orders.")

    # Simulate a delay to mimic a real-world scenario
    time.sleep(1)

    # Close both cursors
    cursor1.close()
    cursor2.close()

    # Return the connection to the pool
    connection_pool.putconn(conn)

# Create multiple threads to run queries concurrently
threads = []
for i in range(5):
    t = threading.Thread(target=query_with_pool_and_multiple_cursors, args=(i,))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

print("All threads have finished execution.")


Thread-0 executing query...
Thread-0 fetched 5 rows from users and 5 rows from orders.
Thread-1 executing query...
Thread-3 executing query...
Thread-1 fetched 5 rows from users and 5 rows from orders.
Thread-3 fetched 5 rows from users and 5 rows from orders.
Thread-4 executing query...
Thread-4 fetched 5 rows from users and 5 rows from orders.
Thread-2 executing query...
Thread-2 fetched 5 rows from users and 5 rows from orders.
All threads have finished execution.
